# SQL запросы

**Контекст задачи:**\
Онлайн-маркетплейс Заберу позволяет пользователям со всей России приобрести товары ручной работы от небольших производителей. Маркетплейс проводит рекламные акции длительностью 1 день. Маркетплейс хочет узнать насколько эффективны рекламные кампании.

**Источник данных:**\
По ссылке   https://yadi.sk/d/pAWR5UW4qhpJSg файлы с описанием таблиц и параметрами для подключения к базе данных. Создайте архив с итоговыми запросами и результатами и отправьте его в тренажере.

**Задание:**
- Посчитайте затраты на каждый рекламный источник.
- Посчитайте суммарную выручку, максимальную и минимальную выручку от заказов, сделанных с настольных ПК (тип устройства desktop) пользователями, которые хотя бы раз перешли из рекламного источника, на который было потрачено больше всего денег. Сделайте выводы.
- Введём новую метрику эффективности рекламы — дневные затраты на рекламу на пользователя по источнику. Она рассчитывается для каждого отдельного дня и равна объёму денежных средств, которые были израсходованы на то, чтобы пользователь перешёл по рекламе на сайт из данного рекламного источника. Иными словами, метрика для каждого дня равна отношению рекламных затрат по данному источнику к количеству посетителей, перешедших на сайт с данного источника.\
Рассчитайте для каждого дня введённую метрику.

Считайте, что целевое действие было произведено тогда, когда сессия началась. Например, если посетитель провёл на сайте несколько минут с 23:50 14 апреля по 0:20 15 апреля, то его нужно включать в расчёт за 14 апреля.

**Описание данных:**\
Таблица visits (лог сервера с информацией о посещениях сайта):
- Uid — уникальный идентификатор пользователя
- Device — категория устройства пользователя
- Start Ts — дата и время начала сессии
- End Ts — дата и время окончания сессии
- Source Id — идентификатор рекламного источника, из которого пришел пользователь

Таблица orders (информация о заказах):
- Uid — уникальный пользователя, который сделал заказ
- Buy Ts — дата и время заказа
- Revenue — выручка Яндекс.Афиши с этого заказа

Таблица advertisment_costs (информация о затратах на маркетинг):
- source_id — идентификатор рекламного источника
- dt — дата
- costs — затраты на этот рекламный источник в этот день

In [1]:
# установка библиотеки для доступа к PostgreSQL
!pip install psycopg2

In [2]:
# Импорт библиотек
import sys
import getopt
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# настройка окружения
pd.options.display.max_columns=None
pd.options.display.float_format='{:.2f}'.format

## Подключение к БД

In [4]:
# Параметры подключения к БД

db_config = {'user': 'analyst',         # имя пользователя
             'pwd': 'I80qcMa',          # пароль
             'host': 'logdb.cdxwh80nuzgz.us-west-2.rds.amazonaws.com', # адрес сервера
             'port': 5432,              # порт подключения
             'db': 'shopinfo'}          # название базы данных

# Строка соединения с БД
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                        db_config['pwd'],
                                                        db_config['host'],
                                                        db_config['port'],
                                                        db_config['db'])

In [5]:
# Подключаемся к БД.
engine = create_engine(connection_string)

# Выполняем запрос и сохраняем результат
# выполнения в DataFrame

# Формируем sql-запросы для первой оценки данных
# загружаем только первые 50 строк
query_vizit = '''SELECT *
        FROM visits 
        '''

query_orders = '''SELECT *
        FROM orders 
        LIMIT 50'''

query_advertisment_costs = '''SELECT *
        FROM advertisment_costs 
        LIMIT 50'''
    
df_visits = pd.io.sql.read_sql(query_vizit, con = engine, index_col = 'id')
df_orders = pd.io.sql.read_sql(query_orders, con = engine, index_col = 'id')
df_advert = pd.io.sql.read_sql(query_advertisment_costs, con = engine, index_col = 'id')

#### Визульная оценка данных в таблицах

In [6]:
df_visits.head(3)

,uid,device,sourceid,startts,endts
id,,,,,
0,16879256277536000000.00,touch,4,2017-12-20 17:20:00,2017-12-20 17:38:00
1,104060357244891744.00,desktop,2,2018-02-19 16:53:00,2018-02-19 17:21:00
2,7459035603376831488.00,touch,5,2017-07-01 01:54:00,2017-07-01 01:54:00


In [7]:
df_orders.head(3)

,uid,buyts,revenue
id,,,
14336,10329302124590700544.00,2017-06-01 00:10:00,$17.00
14337,11627257723692900352.00,2017-06-01 00:25:00,$0.55
14338,17903680561304199168.00,2017-06-01 00:27:00,$0.37


In [8]:
df_advert.head(3)

,source_id,dt,costs
id,,,
1,1,2017-06-01,$75.20
2,1,2017-06-02,$62.25
3,1,2017-06-03,$36.53


In [9]:
# Получение информации о типе данных в таблицах БД
# 
query_data_type = '''
    SELECT table_name, column_name, data_type 
    FROM INFORMATION_SCHEMA.COLUMNS 
    WHERE table_name = 'visits' OR table_name = 'orders' OR table_name = 'advertisment_costs';
    '''
# выполнение запроса к БД
df_query_1 = pd.io.sql.read_sql(query_data_type, con = engine)
# вывод результата
df_query_1.sort_values(by='table_name')

,table_name,column_name,data_type
4,advertisment_costs,id,integer
5,advertisment_costs,source_id,integer
6,advertisment_costs,dt,text
7,advertisment_costs,costs,money
0,orders,id,integer
1,orders,uid,numeric
2,orders,buyts,timestamp without time zone
3,orders,revenue,money
8,visits,id,integer
9,visits,uid,numeric


**Вывод:** в таблице  'advertisment_costs' поле 'dt' имеет тип 'text' вместо 'timestamp', нужно учитывать при составлении запроса.

## Решение основного задания

#### ЗАДАНИЕ 1
Посчитайте затраты на каждый рекламный источник.

In [10]:
# Задание 1
# SQL запрос - ВАРИАНТ 1 (сортировка средствами SQL)

query_1 = '''
          SELECT source_id, SUM(costs) AS sum_costs
          FROM advertisment_costs
          GROUP BY source_id
          ORDER BY sum_costs DESC
       '''
# выполнение запроса к БД
df_query_1 = pd.io.sql.read_sql(query_1, con = engine)
# вывод результата
df_query_1

,source_id,sum_costs
0,3,"$141,321.63"
1,4,"$61,073.60"
2,5,"$51,757.10"
3,2,"$42,806.04"
4,1,"$20,833.27"
5,10,"$5,822.49"
6,9,"$5,517.49"


In [11]:
# Задание 1
# SQL запрос  - ВАРИАНТ 2 (переобразование данных MONEY в NUMERIC и сортировка средствами Pandas)

query_1 = '''
          SELECT source_id, SUM(CAST(costs AS NUMERIC(12,2))) AS sum_costs
          FROM advertisment_costs
          GROUP BY source_id
       '''
# выполнение запроса к БД
df_query_1 = pd.io.sql.read_sql(query_1, con = engine)
# вывод результата
df_query_1.sort_values(by='sum_costs', ascending=False)

,source_id,sum_costs
6,3,141321.63
4,4,61073.60
3,5,51757.10
5,2,42806.04
2,1,20833.27
0,10,5822.49
1,9,5517.49


**Вывод:** Наибольшие затраты были на источник 3, наименьшие на источник 9.

#### ЗАДАНИЕ 2
Посчитайте суммарную выручку, максимальную и минимальную выручку от заказов, сделанных с настольных ПК (тип устройства desktop) пользователями, которые хотя бы раз перешли из рекламного источника, на который было потрачено больше всего денег. Сделайте выводы.

In [99]:
# Задание 2
# SQL запрос
# конструкция местами избыточна, предохраняет от изменения формата даты в таблицах
# При построении запроса проверяется условие начала сессии как указано в 3 задании.
# То есть: если посетитель зашел на сайт в 23:50 14 апреля, а покупку совершил во время этой сессии в 0:20 15 апреля,
# то он включен в расчёт за 14 апреля.
# Официального разъяснения в Slack по этому вопросу на момент создания не получил, поэтом успользовал этот алгоритм.

query_2 = '''
    SELECT 
        DATE_TRUNC('day', SQ_revenue.startts) AS startts , 
        SUM(CAST(SQ_revenue.revenue AS NUMERIC(12,2))) AS SUM,
        MIN(CAST(SQ_revenue.revenue AS NUMERIC(12,2))) AS MIN,
        MAX(CAST(SQ_revenue.revenue AS NUMERIC(12,2))) AS MAX
    FROM 
        (SELECT uid, sourceid, startts, endts, revenue.revenue
         FROM visits 
             LEFT JOIN 
            (SELECT revenue, buyts
             FROM orders) AS revenue
         ON revenue.buyts BETWEEN visits.startts AND visits.endts
         WHERE revenue.revenue IS NOT NULL
        ) AS SQ_revenue
    
    LEFT JOIN
    
        (SELECT uid, sourceid, DATE_TRUNC('day', startts) AS startts
        FROM visits 
             INNER JOIN
            (SELECT dt, source_id, costs 
             FROM advertisment_costs
             WHERE costs IN
               (SELECT MAX(costs) AS costs
                FROM advertisment_costs
                GROUP BY DATE_TRUNC('day', CAST(dt AS DATE)) 
                )) AS source
             ON source.source_id = visits.sourceid 
             AND DATE_TRUNC('day', CAST(source.dt AS DATE)) = DATE_TRUNC('day', startts)
        WHERE device='desktop') AS SQ_source
   
   ON SQ_revenue.uid = SQ_source.uid
   
   WHERE SQ_source.sourceid IS NOT NULL
   GROUP BY DATE_TRUNC('day', SQ_revenue.startts)
   ORDER BY startts;
        '''
# выполнение запроса к БД
df_query_2 = pd.io.sql.read_sql(query_2, con = engine)
# вывод результата
display(df_query_2.head(10))

,startts,sum,min,max
0,2017-06-01,32.59,0.24,10.00
1,2017-06-02,14.22,0.30,3.67
2,2017-06-03,0.49,0.49,0.49
3,2017-06-04,23.95,0.15,17.11
4,2017-06-05,38.97,0.30,6.35
5,2017-06-06,356.30,0.15,31.17
6,2017-06-07,113.83,0.15,24.44
7,2017-06-08,105.21,0.18,44.00
8,2017-06-09,134.60,1.04,18.33
9,2017-06-10,3.05,3.05,3.05


In [100]:
# общее описание полученных данных
df_query_2.describe()

,sum,min,max
count,353.00,353.00,353.00
mean,243.44,0.60,38.21
std,429.20,0.75,75.04
min,0.49,0.00,0.49
25%,53.53,0.18,11.82
50%,131.66,0.37,18.33
75%,296.57,0.61,35.44
max,6085.01,6.72,856.78


**Вывод:**\
Предполагаю, что здесь не ожидается какого-либо развернутого анализа с полученным датасетом, поэтому вывод будет кратким.\
Выручка от самого дорогого дневного источника рекламы имеет всплески до 6085 у.е. в день, при этом в среднем составляет 243 и медиана 131. Средний разброс минимум-максимум выручки с одного клиента: 0.60-38.21 у.е.

#### ЗАДАНИЕ 3
Введём новую метрику эффективности рекламы — дневные затраты на рекламу на пользователя по источнику. Она рассчитывается для каждого отдельного дня и равна объёму денежных средств, которые были израсходованы на то, чтобы пользователь перешёл по рекламе на сайт из данного рекламного источника. Иными словами, метрика для каждого дня равна отношению рекламных затрат по данному источнику к количеству посетителей, перешедших на сайт с данного источника.
Рассчитайте для каждого дня введённую метрику.

In [109]:
# Задание 3
# SQL запрос


query_3 = '''
SELECT 
    SQ_visits.startts,
    SQ_visits.sourceid,
    CAST(SQ_advert.costs / SQ_visits.count AS NUMERIC(12,2))  AS unit_cost
        
FROM
    (SELECT DATE_TRUNC('day', startts) AS startts, sourceid, COUNT(DISTINCT uid) AS count
    FROM visits 
    GROUP BY DATE_TRUNC('day', startts), sourceid) AS SQ_visits
    
    LEFT JOIN
    
    (SELECT DATE_TRUNC('day', CAST(dt AS DATE)) AS dt, source_id, SUM(costs) AS costs
    FROM advertisment_costs
    GROUP BY DATE_TRUNC('day', CAST(dt AS DATE)), source_id) AS SQ_advert
    
    ON SQ_visits.startts = SQ_advert.dt AND SQ_visits.sourceid = SQ_advert.source_id
    '''

# выполнение запроса к БД
df_query_3 = pd.io.sql.read_sql(query_3, con = engine)
# вывод результата
df_query_3.sort_values(by='startts').head(10)

,startts,sourceid,unit_cost
0,2017-06-01,1,10.74
1,2017-06-01,2,7.36
2,2017-06-01,3,24.95
3,2017-06-01,4,7.67
4,2017-06-01,5,8.13
5,2017-06-01,9,3.26
6,2017-06-01,10,7.54
7,2017-06-02,1,20.75
8,2017-06-02,2,26.47
9,2017-06-02,3,14.96


# ЭКСПЕРИМЕНТЫ

In [105]:
# Задание 3
# SQL запрос


query_3 = '''

    
    (SELECT DATE_TRUNC('day', CAST(dt AS DATE)) AS dt, source_id, SUM(costs) AS costs
    FROM advertisment_costs
    GROUP BY DATE_TRUNC('day', CAST(dt AS DATE)), source_id) 
    '''

# выполнение запроса к БД
df_query_3 = pd.io.sql.read_sql(query_3, con = engine)
# вывод результата
df_query_3#.sort_values(by='startts').head(10)

,dt,source_id,costs
0,2017-12-25 00:00:00+00:00,4,$304.38
1,2017-08-25 00:00:00+00:00,10,$4.29
2,2018-05-08 00:00:00+00:00,3,$267.01
3,2017-09-11 00:00:00+00:00,2,$82.77
4,2018-03-29 00:00:00+00:00,1,$40.00
...,...,...,...
2537,2017-09-20 00:00:00+00:00,5,$105.49
2538,2017-11-30 00:00:00+00:00,2,$143.42
2539,2017-09-05 00:00:00+00:00,2,$96.44
2540,2018-03-09 00:00:00+00:00,9,$14.39


In [47]:
from datetime import datetime
datetime.strptime('2017-06-01', '%Y-%m-%d')

datetime.datetime(2017, 6, 1, 0, 0)

In [61]:
import numpy as np
df_visits['date'] = df_visits['startts'].dt.date 
df_visits.loc[(df_visits['sourceid']==3) & (df_visits['device'] == 'desktop')].sort_values(by='date').head(15)

,uid,device,sourceid,startts,endts,date
id,,,,,,
13102,17415680957395800064.00,desktop,3,2017-06-01 14:25:00,2017-06-01 14:31:00,2017-06-01
17931,7397096008106280960.00,desktop,3,2017-06-01 10:39:00,2017-06-01 10:40:00,2017-06-01
5659,5200639919992470528.00,desktop,3,2017-06-01 19:08:00,2017-06-01 19:25:00,2017-06-01
12587,958834170939882752.00,desktop,3,2017-06-01 18:33:00,2017-06-01 18:41:00,2017-06-01
12866,3133331625418050560.00,desktop,3,2017-06-01 20:44:00,2017-06-01 20:45:00,2017-06-01
17565,14495730715166099456.00,desktop,3,2017-06-01 12:53:00,2017-06-01 13:07:00,2017-06-01
24143,15894956789369800704.00,desktop,3,2017-06-01 13:12:00,2017-06-01 13:17:00,2017-06-01
22574,16269647965011599360.00,desktop,3,2017-06-01 11:09:00,2017-06-01 11:16:00,2017-06-01
4121,4267975796962764800.00,desktop,3,2017-06-01 00:38:00,2017-06-01 00:41:00,2017-06-01


In [87]:
# Задание 2
# SQL запрос
# конструкция местами избыточна, предохраняет от изменения формата даты в таблицах

query_2 = '''
SELECT  DATE_TRUNC('day', SUB.startts) AS startts , SUM(SUB.revenue)
FROM
    (SELECT uid, sourceid, startts, endts, revenue.revenue AS revenue, device
    FROM visits LEFT JOIN 
        (SELECT revenue, buyts
         FROM orders) AS revenue
        ON revenue.buyts BETWEEN visits.startts AND visits.endts
    WHERE revenue.revenue IS NOT NULL   AND  sourceid = 3 AND DATE_TRUNC('day', startts) = '2017-06-03'
    AND device = 'desktop'
     ) AS SUB 
GROUP BY DATE_TRUNC('day', SUB.startts)
       '''
# выполнение запроса к БД
df_query_2 = pd.io.sql.read_sql(query_2, con = engine)
# вывод результата
df_query_2.sort_values(by='startts')

,startts,sum
0,2017-06-03,$0.49


In [16]:
# Задание 2
# SQL запрос
# конструкция местами избыточна, предохраняет от изменения формата даты в таблицах

query_2 = '''
    
        (SELECT uid, sourceid, DATE_TRUNC('day', startts) AS startts
        FROM visits INNER JOIN
           (SELECT dt, source_id, costs 
           FROM advertisment_costs
           WHERE costs IN
               (SELECT MAX(costs) AS costs
                FROM advertisment_costs
                GROUP BY DATE_TRUNC('day', CAST(dt AS DATE)) 
                )) AS source
           ON source.source_id = visits.sourceid 
           AND DATE_TRUNC('day', CAST(source.dt AS DATE)) = DATE_TRUNC('day', startts)
        WHERE device='desktop') 
   
       '''
# выполнение запроса к БД
df_query_2 = pd.io.sql.read_sql(query_2, con = engine)
# вывод результата
df_query_2.sort_values(by='startts')

,uid,sourceid,startts
40,14495730715166099456.00,3,2017-06-01
33,10241525901117399040.00,3,2017-06-01
34,17415680957395800064.00,3,2017-06-01
35,958834170939882752.00,3,2017-06-01
36,18271362086313799680.00,3,2017-06-01
...,...,...,...
4510,14419274054660499456.00,3,2018-05-31
4511,12517926191595100160.00,3,2018-05-31
4512,18075397354178899968.00,3,2018-05-31
4514,18164454063411499008.00,3,2018-05-31
